# How many games can DVOA pick correctly?
There are a few measures of DVOA given by Football Outsiders (DVOA, DAVE, weighted DVOA, and position-based DVOA). For the first test I'll see how many times DVOA can correctly pick the winner of a football game throughout the season. Then I'll look at other measures (and combinations of these measures) to see if they do a better job. We will find out if DVOA is a competitive guesser in a straight-up pick-em pool. 

Finally, the results from picking game winners using all varieties of DVOA will be saved to a file for later use. 

DVOA Results are taken from the weekly DVOA reports published on Football Outsiders: https://www.footballoutsiders.com/dvoa-ratings

NOTE:
On the Football Outsiders' website, they publish using titles with the format "Week 1 DVOA Ratings". These are actually the ratings computed after week 1 games have been played. Because I am interested in making predictions I only use data that is available before games are played each week. I use the published preseason DVOA ratings to predict outcomes of week 1 games, published week 1 ratings to predict outcomes of week 2, etc.

In [148]:
## Imports for data processing and plotting
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
%matplotlib inline

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 5000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Plot settings
plt.rcParams['xtick.major.width'] = .9
plt.rcParams['xtick.major.size'] = 8.
plt.rcParams['xtick.minor.size'] = 5.
plt.rcParams['xtick.minor.width'] = .9
plt.rcParams['xtick.labelsize'] = 19.
plt.rcParams['ytick.major.width'] = .9
plt.rcParams['ytick.major.size'] = 8.
plt.rcParams['ytick.minor.size'] = 5.
plt.rcParams['ytick.minor.width'] = .9
plt.rcParams['ytick.labelsize'] = 19.
plt.rcParams['axes.linewidth'] = 2.
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.labelsize'] = 20.
matplotlib.rcParams['agg.path.chunksize'] = 10000
col1 = cm.viridis(np.linspace(0,0.8,13))
col2 = cm.Set2(np.linspace(0,1,8))
col3 = cm.Set3(np.linspace(0,1,12))

In [149]:
indir = '/Users/sydney/Desktop/data_science/NFL_pickem_DVOA/'
outdir = indir

## Read in results from NFL 2017 and 2018 regular season matchups
NFL results are taken from: https://www.pro-football-reference.com/years/2017/games.htm

In [150]:
nfl_results17 = pd.read_csv(indir + 'NFL_results_data/2017_NFL_results.csv')
nfl_results18 = pd.read_csv(indir + 'NFL_results_data/2018_NFL_results.csv')

In [151]:
##NFL results have long team names, switch them to the short abbreviations - 2017 Season
teamName_short17 = np.asarray(['LARM','BAL','BUF','KC','CAR','DAL','PIT',
                  'DET','ATL','GB','JAC','PHI','MIN','OAK',
                  'TEN','MIA','TB','LACH','DEN','CHI','WAS',
                  'NO','SEA','NE','NYG','HOU','CLE','ARI',
                  'SF','NYJ','CIN','IND'])

teamName_long17 = np.asarray(['Los Angeles Rams', 'Baltimore Ravens', 'Buffalo Bills', 'Kansas City Chiefs',
                'Carolina Panthers', 'Dallas Cowboys', 'Pittsburgh Steelers', 'Detroit Lions',
                'Atlanta Falcons', 'Green Bay Packers', 'Jacksonville Jaguars', 'Philadelphia Eagles',
                'Minnesota Vikings', 'Oakland Raiders', 'Tennessee Titans', 'Miami Dolphins',
                'Tampa Bay Buccaneers', 'Los Angeles Chargers', 'Denver Broncos', 'Chicago Bears',
                'Washington Redskins', 'New Orleans Saints', 'Seattle Seahawks', 'New England Patriots',
                'New York Giants', 'Houston Texans', 'Cleveland Browns', 'Arizona Cardinals',
                'San Francisco 49ers', 'New York Jets', 'Cincinnati Bengals', 'Indianapolis Colts'])

##NFL results have long team names, switch them to the short abbreviations - 2018 Season
teamName_short18 = np.asarray(['LAR','BAL','BUF','KC','CAR','DAL','PIT',
                  'DET','ATL','GB','JAX','PHI','MIN','OAK',
                  'TEN','MIA','TB','LAC','DEN','CHI','WAS',
                  'NO','SEA','NE','NYG','HOU','CLE','ARI',
                  'SF','NYJ','CIN','IND'])

teamName_long18 = np.asarray(['Los Angeles Rams', 'Baltimore Ravens', 'Buffalo Bills', 'Kansas City Chiefs',
                'Carolina Panthers', 'Dallas Cowboys', 'Pittsburgh Steelers', 'Detroit Lions',
                'Atlanta Falcons', 'Green Bay Packers', 'Jacksonville Jaguars', 'Philadelphia Eagles',
                'Minnesota Vikings', 'Oakland Raiders', 'Tennessee Titans', 'Miami Dolphins',
                'Tampa Bay Buccaneers', 'Los Angeles Chargers', 'Denver Broncos', 'Chicago Bears',
                'Washington Redskins', 'New Orleans Saints', 'Seattle Seahawks', 'New England Patriots',
                'New York Giants', 'Houston Texans', 'Cleveland Browns', 'Arizona Cardinals',
                'San Francisco 49ers', 'New York Jets', 'Cincinnati Bengals', 'Indianapolis Colts'])

In [152]:
### Loop through and correct 2017 Names
winner_short17 = []
loser_short17 = []

for n in range(len(nfl_results17['Winner/tie'])):
    ## Winners
    loc = np.where(teamName_long17 == nfl_results17['Winner/tie'][n])[0]
    winner_short17 = np.append(winner_short17, teamName_short17[loc])
    ## Losers
    loc = np.where(teamName_long17 == nfl_results17['Loser/tie'][n])[0]
    loser_short17 = np.append(loser_short17, teamName_short17[loc])

### Loop through and correct 2018 Names
winner_short18 = []
loser_short18 = []

for n in range(len(nfl_results18['Winner/tie'])):
    ## Winners
    loc = np.where(teamName_long18 == nfl_results18['Winner/tie'][n])[0]
    winner_short18 = np.append(winner_short18, teamName_short18[loc])
    ## Losers
    loc = np.where(teamName_long18 == nfl_results18['Loser/tie'][n])[0]
    loser_short18 = np.append(loser_short18, teamName_short18[loc])

In [153]:
### Make new dataframes with only the info you would have before the game (team names, week)
## 2017 Season
games17_info = {'Week': nfl_results17['Week'],
               'Winner':winner_short17,
               'Loser':loser_short17}
games17 = pd.DataFrame(games17_info)

## 2018 Season
games18_info = {'Week': nfl_results18['Week'],
               'Winner':winner_short18,
               'Loser':loser_short18}
games18 = pd.DataFrame(games18_info)

## Count DVOA correct picks
Loop through each week in the season, open the DVOA predictions file for that week, for each matchup find who DVOA would pick, award one point for correct pick, no points for incorrect pick

In [154]:
## Number of weeks in NFL regular season
nweeks = 17

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [155]:
## Save output to more descriptive names
dvoaOnly_correctPickCount17 = correctPickCount17
dvoaOnly_tiedPicks17 = tiedPicks17
dvoaOnly_weeklyWins17 = weeklyWins17

In [156]:
## Number of weeks in NFL regular season
nweeks = 17

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [157]:
dvoaOnly_correctPickCount18 = correctPickCount18
dvoaOnly_tiedPicks18 = tiedPicks18
dvoaOnly_weeklyWins18 = weeklyWins18

## Using DAVE for predicting weeks 2-8
DAVE is a success measure used by Football Outsiders in the first two months of the season (computed following week 1 and available for predicting week 2). This measure starts by giving high weight to preseason prediction (90% in week 1) and this weight is reduced each week until week 8. This gives a more informed success rating for teams in the first half of the season when a team has limited historical data. 

After week 8, regular total DVOA will be used for prediction. 

In [158]:
## Number of weeks in NFL regular season
nweeks = 17
dave_min_week = 1 ## First published in week 1 to predict for week 2
dave_max_week = 7 ## Last published in week 7 to predict for week 8

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        ## If it's the first week of the season or after the 8th week, use DVOA
        if (w < dave_min_week) | (w > dave_max_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        ## If it's the 2nd-8th week, use DAVE
        else:
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DAVE'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DAVE'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [159]:
daveTOdvoa_correctPickCount17 = correctPickCount17
daveTOdvoa_tiedPicks17 = tiedPicks17
daveTOdvoa_weeklyWins17 = weeklyWins17

In [160]:
## Number of weeks in NFL regular season
nweeks = 17
dave_min_week = 1 ## First published in week 1 to predict for week 2
dave_max_week = 8 ## Last published in week 8 to predict for week 9

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        ## If it's the first week of the season or after the 8th week, use DVOA
        if (w < dave_min_week) | (w > dave_max_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        ## If it's the 2nd-8th week, use DAVE
        else:
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DAVE'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DAVE'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [161]:
daveTOdvoa_correctPickCount18 = correctPickCount18
daveTOdvoa_tiedPicks18 = tiedPicks18
daveTOdvoa_weeklyWins18 = weeklyWins18

## Using Weighted DVOA for predicting weeks 9-17
Weigted DVOA is available in the second half of the season and gives higher weight to more recent games played. For weeks 1 - 7 I will use total DVOA for predicting and in weeks 8 - 17 I will use the weighted DVOA. 

In [162]:
## Number of weeks in NFL regular season
nweeks = 17
wdvoa_min_week = 8 ## First published in week 8 to predict for week 9

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        ## If it's the first half of the season use DVOA
        if (w < wdvoa_min_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        ## If it's the 2nd half of the season, use weighted DVOA
        else:
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['WDVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['WDVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [163]:
dvoaTOwdvoa_correctPickCount17 = correctPickCount17
dvoaTOwdvoa_tiedPicks17 = tiedPicks17
dvoaTOwdvoa_weeklyWins17 = weeklyWins17

In [164]:
## Number of weeks in NFL regular season
nweeks = 17
wdvoa_min_week = 9 ## First published in week 9 to predict for week 10

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        ## If it's the first half of the season use DVOA
        if (w < wdvoa_min_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        ## If it's the 2nd half of the season, use weighted DVOA
        else:
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['WDVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['WDVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [165]:
dvoaTOwdvoa_correctPickCount18 = correctPickCount18
dvoaTOwdvoa_tiedPicks18 = tiedPicks18
dvoaTOwdvoa_weeklyWins18 = weeklyWins18

## DAVE first, then weighted DVOA
In week 1, use DVOA to predict (it's the only predictor available). Then in the first half of the season use DAVE and in the second half of the season use weighted DVOA. 

In [166]:
## Number of weeks in NFL regular season
nweeks = 17
wdvoa_min_week = 8 ## First published in week 8 to predict for week 9
dave_min_week = 1 ## First week that DAVE is published

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        ## If it's the first week of the season use DVOA
        if (w < dave_min_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        ## If it's the 1st half of the season, use DAVE
        elif (w >= dave_min_week) & (w < wdvoa_min_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DAVE'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DAVE'][dvoa_idx_l]
        ## If it's the 2nd half of the season, use weighted DVOA
        else:
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['WDVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['WDVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [167]:
daveTOwdvoa_correctPickCount17 = correctPickCount17
daveTOwdvoa_tiedPicks17 = tiedPicks17
daveTOwdvoa_weeklyWins17 = weeklyWins17

In [168]:
## Number of weeks in NFL regular season
nweeks = 17
wdvoa_min_week = 9 ## First published in week 9 to predict for week 10
dave_min_week = 1 ## First week that DAVE is published

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        ## If it's the first week of the season use DVOA
        if (w < dave_min_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DVOA'][dvoa_idx_l]
        ## If it's the 1st half of the season, use DAVE
        elif (w >= dave_min_week) & (w < wdvoa_min_week):
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['TOTAL DAVE'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['TOTAL DAVE'][dvoa_idx_l]
        ## If it's the 2nd half of the season, use weighted DVOA
        else:
            dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
            dvoa_val_w = dvoa_data['WDVOA'][dvoa_idx_w]

            dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
            dvoa_val_l = dvoa_data['WDVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [169]:
daveTOwdvoa_correctPickCount18 = correctPickCount18
daveTOwdvoa_tiedPicks18 = tiedPicks18
daveTOwdvoa_weeklyWins18 = weeklyWins18

## Use offensive DVOA for all weeks
DVOA can be broken down for offense, defense, and special teams. Use the offense success value here to predict winners

In [170]:
## Number of weeks in NFL regular season
nweeks = 17

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['OFF. DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['OFF. DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [171]:
offdvoa_correctPickCount17 = correctPickCount17
offdvoa_tiedPicks17 = tiedPicks17
offdvoa_weeklyWins17 = weeklyWins17

In [172]:
## Number of weeks in NFL regular season
nweeks = 17

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['OFF. DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['OFF. DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [173]:
offdvoa_correctPickCount18 = correctPickCount18
offdvoa_tiedPicks18 = tiedPicks18
offdvoa_weeklyWins18 = weeklyWins18

## Use defensive DVOA for all weeks
DVOA can be broken down for offense, defense, and special teams. Use the defense success value here to predict winners. Note that a defense is better when their DVOA is smaller. 

In [174]:
## Number of weeks in NFL regular season
nweeks = 17

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['DEF. DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['DEF. DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w < dvoa_val_l): ## Smaller defense DVOA is better
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [175]:
defdvoa_correctPickCount17 = correctPickCount17
defdvoa_tiedPicks17 = tiedPicks17
defdvoa_weeklyWins17 = weeklyWins17

In [176]:
## Number of weeks in NFL regular season
nweeks = 17

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['DEF. DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['DEF. DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w < dvoa_val_l): ## Smaller defense DVOA is better
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [177]:
defdvoa_correctPickCount18 = correctPickCount18
defdvoa_tiedPicks18 = tiedPicks18
defdvoa_weeklyWins18 = weeklyWins18

## Use Special Teams DVOA for all weeks
DVOA can be broken down for offense, defense, and special teams. Use the special teams success value here to predict winners. 

In [178]:
## Number of weeks in NFL regular season
nweeks = 17

### 2017 correct pick counter
correctPickCount17 = 0
tiedPicks17 = 0
weeklyWins17 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2017_FO_DVOA_data/2017_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games17[games17['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games17['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['S.T. DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['S.T. DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount17 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks17 += 1
            
    weeklyWins17[w] = weeklyWinsCounter

In [179]:
stdvoa_correctPickCount17 = correctPickCount17
stdvoa_tiedPicks17 = tiedPicks17
stdvoa_weeklyWins17 = weeklyWins17

In [180]:
## Number of weeks in NFL regular season
nweeks = 17

### 2018 correct pick counter
correctPickCount18 = 0
tiedPicks18 = 0
weeklyWins18 = np.zeros(nweeks)

for w in range(0,nweeks):
    ## Read in DVOA file for this week
    dvoa_data = pd.read_csv(indir + '2018_FO_DVOA_data/2018_dvoa_wk'+str(w)+'.csv')
    
    ## Which NFL games were played this week
    gamesThisWeek = games18[games18['Week'] == w+1]
    weeklyWinsCounter = 0
    
    ## Loop through games
    idx = np.where(games18['Week'] == w+1)[0]
    for g in range(len(idx)):
        wTeam = gamesThisWeek['Winner'][idx[g]]
        lTeam = gamesThisWeek['Loser'][idx[g]]
        
        dvoa_idx_w = np.where(dvoa_data['TEAM'] == wTeam)[0][0]
        dvoa_val_w = dvoa_data['S.T. DVOA'][dvoa_idx_w]
        
        dvoa_idx_l = np.where(dvoa_data['TEAM'] == lTeam)[0][0]
        dvoa_val_l = dvoa_data['S.T. DVOA'][dvoa_idx_l]
        
        if (dvoa_val_w > dvoa_val_l):
            correctPickCount18 += 1
            weeklyWinsCounter += 1
            
        if (dvoa_val_w == dvoa_val_l):
            tiedPicks18 += 1
            
    weeklyWins18[w] = weeklyWinsCounter

In [181]:
stdvoa_correctPickCount18 = correctPickCount18
stdvoa_tiedPicks18 = tiedPicks18
stdvoa_weeklyWins18 = weeklyWins18

# Take all of these weekly wins counts, totals, and ties and save them to files

In [182]:
### Weekly wins file
week_arr = np.arange(1,18,1)

## 2017 data
weekly_data17 = {'Week':week_arr,
                'DVOA Only': dvoaOnly_weeklyWins17,
                'DAVE to DVOA': daveTOdvoa_weeklyWins17,
                'DVOA to WDVOA': dvoaTOwdvoa_weeklyWins17,
                'DAVE to WDVOA': daveTOwdvoa_weeklyWins17,
                'OFF DVOA': offdvoa_weeklyWins17,
                'DEF DVOA': defdvoa_weeklyWins17,
                'ST DVOA': stdvoa_weeklyWins17}
weeklyWinsData17 = pd.DataFrame(weekly_data17)
weeklyWinsData17.to_csv(outdir + '2017_FO_DVOA_data/winsByWeek_2017.csv')

## 2018 data
weekly_data18 = {'Week':week_arr,
                'DVOA Only': dvoaOnly_weeklyWins18,
                'DAVE to DVOA': daveTOdvoa_weeklyWins18,
                'DVOA to WDVOA': dvoaTOwdvoa_weeklyWins18,
                'DAVE to WDVOA': daveTOwdvoa_weeklyWins18,
                'OFF DVOA': offdvoa_weeklyWins18,
                'DEF DVOA': defdvoa_weeklyWins18,
                'ST DVOA': stdvoa_weeklyWins18}
weeklyWinsData18 = pd.DataFrame(weekly_data18)
weeklyWinsData18.to_csv(outdir + '2018_FO_DVOA_data/winsByWeek_2018.csv')

In [183]:
### Totals and ties (for simplicity)
totalTypes = ['Total Correct', 'Ties']

## 2017 data
wins_data17 = {'TotalType':totalTypes,
                'DVOA Only': [dvoaOnly_correctPickCount17,dvoaOnly_tiedPicks17],
                'DAVE to DVOA': [daveTOdvoa_correctPickCount17,daveTOdvoa_tiedPicks17],
                'DVOA to WDVOA': [dvoaTOwdvoa_correctPickCount17,dvoaTOwdvoa_tiedPicks17],
                'DAVE to WDVOA': [daveTOwdvoa_correctPickCount17,daveTOwdvoa_tiedPicks17],
                'OFF DVOA': [offdvoa_correctPickCount17,offdvoa_tiedPicks17],
                'DEF DVOA': [defdvoa_correctPickCount17,defdvoa_tiedPicks17],
                'ST DVOA': [stdvoa_correctPickCount17,stdvoa_tiedPicks17]}
winsData17 = pd.DataFrame(wins_data17)
winsData17.to_csv(outdir + '2017_FO_DVOA_data/totalWins_2017.csv')

## 2018 data
wins_data18 = {'TotalType':totalTypes,
                'DVOA Only': [dvoaOnly_correctPickCount18,dvoaOnly_tiedPicks18],
                'DAVE to DVOA': [daveTOdvoa_correctPickCount18,daveTOdvoa_tiedPicks18],
                'DVOA to WDVOA': [dvoaTOwdvoa_correctPickCount18,dvoaTOwdvoa_tiedPicks18],
                'DAVE to WDVOA': [daveTOwdvoa_correctPickCount18,daveTOwdvoa_tiedPicks18],
                'OFF DVOA': [offdvoa_correctPickCount18,offdvoa_tiedPicks18],
                'DEF DVOA': [defdvoa_correctPickCount18,defdvoa_tiedPicks18],
                'ST DVOA': [stdvoa_correctPickCount18,stdvoa_tiedPicks18]}
winsData18 = pd.DataFrame(wins_data18)
winsData18.to_csv(outdir + '2018_FO_DVOA_data/totalWins_2018.csv')